# <center>Домашнее задание
# <center>Создаем Википедию
1. Используя подход аналогичный torchvision, сделать свой класс датасета.   
Необязательное д/з:   
1. Поэкспериментировать с разными архитектурами рекурренток: тип ячеек, слои, нормализация, методы оптимизации

In [1]:
import torchtext
from torchtext.data.utils import get_tokenizer
import torch

## <center>1.

In [2]:
class WikiText2Dataset:
    
    def __init__(self, batch_size=128, eval_batch_size=128, sequence_length=30, tokenize_func=list):
        self.tokenize_func = tokenize_func
        self.batch_size = batch_size
        self.eval_batch_size = eval_batch_size
        self.sequence_length = sequence_length
        self.symbol2idx = None
        self.idx2symbol = None
        self.TEXT = None
        self.train_txt, self.val_txt, self.test_txt = None, None, None
        self.make_TextLoader_cls()
    
    def get_train_val_test_txts(self):
        self.TEXT = torchtext.data.Field(tokenize=get_tokenizer(self.tokenize_func),
                                    init_token='<sos>',
                                    eos_token='<eos>',
                                    lower=False)
        self.train_txt, self.val_txt, self.test_txt = torchtext.datasets.WikiText2.splits(self.TEXT)
        self.TEXT.build_vocab(self.train_txt)
        self.symbol2idx = self.TEXT.vocab.stoi
        self.idx2symbol = self.TEXT.vocab.itos
        return self.train_txt, self.val_txt, self.test_txt
    

    def _batchify(self, data_txt, batch_size):
        data = self.TEXT.numericalize([data_txt.examples[0].text])
        # Divide the dataset into bsz parts.
        nbatch = data.size(0) // batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = data.narrow(0, 0, nbatch * batch_size)
        # Evenly divide the data across the bsz batches.
        data = data.view(batch_size, -1).t().contiguous()
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # data = data.to(device)
        return data
    
    def get_train_loader(self):
        if self.train_txt is None:
            self.get_train_val_test_txts()
        train_data = self._batchify(self.train_txt, self.batch_size)
        train_loader = self.TextLoader(train_data, self.sequence_length)
        return train_loader

    def get_val_loader(self):
        if self.val_txt is None:
            self.get_train_val_test_txts()
        val_data = self._batchify(self.val_txt, self.eval_batch_size)
        val_loader = self.TextLoader(val_data, self.sequence_length)
        return val_loader
    
    def get_test_loader(self):
        if self.test_txt is None:
            self.get_train_val_test_txts()
        test_data = self._batchify(self.test_txt, self.eval_batch_size)
        test_loader = self.TextLoader(test_data, self.sequence_length)
        return test_loader
    
    def make_TextLoader_cls(self):
        class TextLoader:
            def __init__(self, batch_data, sequence_length=30):
                self.batch_data = batch_data
                self.sequence_length = sequence_length

            def _get_batch(self, i):
                seq_len = min(self.sequence_length, len(self.batch_data) - 1 - i)
                data = self.batch_data[i:i+seq_len]
                target = self.batch_data[i+1:i+1+seq_len].view(-1)
                return data, target

            def __iter__(self):
                for i in range(0, self.batch_data.size(0) - 1, self.sequence_length):
                    data, targets = self._get_batch(i)
                    yield data, targets

            def __len__(self):
                return self.batch_data.size(0)
        self.TextLoader = TextLoader

## тестируем

In [3]:
dataset = WikiText2Dataset()

In [4]:
%%time
train_loader = dataset.get_train_loader()
val_loader = dataset.get_val_loader()
test_loader = dataset.get_test_loader()

Wall time: 3 s


In [5]:
list(train_loader)[0]

(tensor([[ 4,  5,  6,  ..., 17,  4,  7],
         [ 3, 11,  4,  ..., 11,  6, 23],
         [ 4, 23, 18,  ..., 10,  9,  5],
         ...,
         [41,  4,  4,  ...,  5,  4,  6],
         [ 4, 64,  7,  ...,  4, 15, 13],
         [35, 10, 20,  ..., 31, 10,  5]]),
 tensor([ 3, 11,  4,  ..., 46, 19,  4]))

In [8]:
list(train_loader)[0][0].shape

torch.Size([30, 128])

In [9]:
list(train_loader)[0][1].shape

torch.Size([3840])

In [6]:
list(val_loader)[0]

(tensor([[ 4,  5, 52,  ..., 15, 14,  7],
         [ 3,  4,  4,  ...,  9,  4, 11],
         [ 4, 12, 24,  ...,  8, 19,  4],
         ...,
         [ 4,  6,  8,  ..., 13,  4, 70],
         [47, 11, 20,  ...,  8, 27, 16],
         [ 9, 16, 15,  ...,  4,  4,  5]]),
 tensor([ 3,  4,  4,  ..., 29,  3, 15]))

In [7]:
list(test_loader)[0]

(tensor([[ 4,  6, 14,  ..., 15, 12, 29],
         [ 3,  6, 10,  ..., 15,  4, 16],
         [ 4, 11, 19,  ...,  7, 16,  8],
         ...,
         [ 5, 10, 10,  ...,  4, 11,  6],
         [11,  8,  9,  ..., 27,  5, 23],
         [ 6,  4,  8,  ...,  4,  4, 15]]),
 tensor([ 3,  6, 10,  ...,  3, 37,  5]))